# Визуализация матричных моделей


https://pyprog.pro/

Матричные модели - это общее название моделей для представления которых используются матрицы или проще говоря таблицы. В Seaborn для данных моделей существуют два вида графиков: heatmap() - для создания тепловых карт и clustermap() - для создания кластерных карт. Обе функции создают графики на уровне Axes и в определенных ситуациях могут оказаться крайне полезны.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Тепловые карты


In [ ]:
data = np.random.randint(0, 30, size=(10, 10))
sns.heatmap(data);

Так же как и двумерная гистограмма, тепловая карта состоит из прямоугольных ячеек, но смысл гистограммы заключается в подсчете попаданий точек в каждую ячейку, а в тепловых картах каждая ячейка просто обозначает цветом значение той или инной ячейки в таблице с данными. Выше, я намеренно сделал квадратную тепловую карту, но она может иметь любую форму, в зависимости от того с какими данными вы работаете:



In [ ]:
fig = plt.figure()

ax_1 = fig.add_subplot(1, 2, 1)
ax_2 = fig.add_subplot(1, 2, 2)

data_1 = np.random.randint(0, 30, size=(25, 5))
data_2 = np.random.randint(0, 30, size=(5, 25))

sns.heatmap(data_1, ax=ax_1);
sns.heatmap(data_2, ax=ax_2);

ax_1.set_title('"длинные" данные',
               fontsize=15)
ax_2.set_title('"широкие" данные',
               fontsize=15)


fig.set_figwidth(14)
fig.set_figheight(6)

plt.show()


Тепловые карты больше всего похожи на двумерную столбчатую диаграмму. Эта аналогия станивится более очевидной если взглянуть всего на одну строку в данных и построить для нее эти графики:

In [ ]:
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(14, 7))
gs = gridspec.GridSpec(nrows=2, ncols=10)

ax_1 = fig.add_subplot(gs[0, 0:8])
ax_2 = fig.add_subplot(gs[1, :])

data = np.random.randint(0, 30, 10)

ax_1.bar(range(len(data)), data)
ax_1.set_xlim(-0.5, 9.5)
sns.heatmap(data.reshape(1, 10), annot=True, ax=ax_2);

ax_1.set_title('Столбчатая диаграмма',
               fontsize=15)
ax_2.set_title('Тепловая карта',
               fontsize=15)


fig.set_figwidth(14)
fig.set_figheight(7)

plt.show()

Как видите, в столбчатых диаграммах, значение переменной обозначается высотой столбца, а в тепловой карте - цветом. Кстати, чтобы облегчить восприятие, на каждую цветную ячейку мы добавиди соответствующее числовое значение с помощью параметра annot = True. Преимущества тепловой карты, становятся очевидны если рассмотреть следующий пример. Сначала загрузим данные, которые содержат информацию о количестве перевезенных пассажиров в разбивке по годам и месяцам:

In [ ]:
flights = pd.read_csv('../input/flights-seaborn/flights.csv')
flights.head()

А теперь сделаем сводную таблицу по годам и месяцам:



In [ ]:
flights = flights.pivot("month", "year", "passengers")
flights

А теперь построим тепловую карту для этой таблицы:



In [ ]:
f, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(data=flights,
            annot=True,
            fmt="d",
            cmap="Purples",
            linewidths=.1);

# Многосегментные графики


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Фасетные графики


Если в данных есть числовые значения, но все они могут быть разбиты на подмножества, то самым оптимальным вариантом анализа является их визуализация в контексте этих подмножеств. Для решения данной задачи существует класс FacetGrid, который позволяет выделять подмножества по трем измерениям: строкам, столбцам и оттенкам.

In [ ]:
penguins = pd.read_csv('../input/penguins/penguins.csv')
penguins.head()

Теперь мы можем создать экземпляр класса FacetGrid:



In [ ]:
g = sns.FacetGrid(data=penguins, col='island', row='sex')


В данном случае, экземпляр класса просто настроил область Figure и расположил на ней сетку из областей Axes, но ничего на них не рисует. А вот что бы что-то нарисовать нужно воспользоваться методом map():

In [ ]:
g = sns.FacetGrid(data=penguins, col='island', row='sex')
g.map(plt.hist, 'body_mass_g');

Такой интерфейс может показаться немного странным и даже неудобным, но вся его задача заключается в настройке сетки графиков. И на самом деле все довольно просто: методу map() мы передали функцию библиотеки Matplotlib plt.hist, а далее указали все аргументы в том порядке в котором они передаются данной функции, а точнее всего один - имя столбца таблицы penguins. Мы можем построить и другие типы графиков для тех же данных:

In [ ]:
g = sns.FacetGrid(data=penguins,
                  col='island',    # одна колонка - один остров
                  row='sex',       # пол особи в отдельной строке
                  hue='species')   # цветом выделяем вид пингвинов

g.map(plt.scatter,             # указываем нужную функцию matplotlib
      'body_mass_g',           # значения по оси X
      'flipper_length_mm',     # значения по оси Y
      s=60, alpha=0.3);        # ключевые аргументы функции

# Визуализация парных соотношений


Класс FacetGrid позволяет рассматривать одну или две переменные в контексте их разных подмножеств. Но не менее часто нужно рассмотреть парные отношения всех переменных в наборе данных. Для этих целей можно воспользоваться функцией pairplot():



In [ ]:
sns.pairplot(penguins);


Эта функция очень удобна для быстрой визуализации парных отношений, но она не предоставляет всей гибкости класса PairGrid, который она использует для создания области Figure и сетки подграфиков (областей Axes). Поскольку верхний и нижний треугольники сетки PairGrid являются симметричными, можно задать разные типы графиков для них и для главной диагонали в том числе:

In [ ]:
g = sns.PairGrid(data=penguins, hue='species')
g.map_upper(sns.scatterplot)
g.map_diag(plt.hist, alpha=0.6)
g.map_lower(sns.kdeplot);